# Class-Project - Media Sentiment and VOting Outcomes

## Topic: Anaylzing Swiss Media Coverage and its Relation to Direct Democratic Decisions

**Description:**  
This project explores how Swiss media coverage—through sentiment and topic analysis—relates to the outcomes of national referenda. Using a dataset of approximately 100–120 newspaper articles covering 20 popular initiatives, we apply Natural Language Processing (NLP) techniques to uncover prevailing themes and emotional tone in the media discourse. The results are then compared to the respective voting outcomes to investigate whether patterns of alignment or divergence emerge between public sentiment and media framing.

*Note: This is an exploratory project and does not aim to establish causal relationships.*

---

## 1. Setup: Install and Import Required Packages

Before starting the analysis, we install and import all necessary Python libraries for text preprocessing, sentiment analysis, topic modeling, and visualization.

In [32]:
# install required packages (pandas, requests, beautifulsoup4, tqdm) if not already installed
# import
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm


load data

In [7]:
input_path = "Data_ML.xlsx" 
df = pd.read_excel(input_path)

In [ ]:
# inspect data
df.head()

,vote_id,id,voting_date,initiative,yes_share,result,news_article_URL,publication_date,newspaper_name,text
0,1,1,2011-02-13,Für den Schutz vor Waffengewalt,0.437,Abgelehnt,https://www.20min.ch/story/tell-srebrenica-und...,2011-01-09,20min,NaN
1,1,2,2011-02-13,Für den Schutz vor Waffengewalt,0.437,Abgelehnt,https://www.20min.ch/story/abstimmung-mit-prom...,2011-02-08,20min,NaN
2,1,3,2011-02-13,Für den Schutz vor Waffengewalt,0.437,Abgelehnt,https://www.20min.ch/story/daempfer-fuer-schut...,2011-01-04,20min,NaN
3,1,4,2011-02-13,Für den Schutz vor Waffengewalt,0.437,Abgelehnt,https://www.20min.ch/story/kommission-lehnt-wa...,2011-01-04,20min,NaN
4,1,5,2011-02-13,Für den Schutz vor Waffengewalt,0.437,Abgelehnt,https://www.20min.ch/story/waffenlobby-wittert...,2011-01-13,20min,NaN


Based on.. we randomly selected 20 initiatives over the past 20 years (almost 80% coverage). With the help of AI tools we randomly selected newsarticle from SRF, NZZ and 20min (popular swiss news outlet) which discuss proposed initiatives PRIOR to voting. As NZZ is the only news outlet in the sample requireing an Account, scrapping its links did not work, therefore we did it manually. 
In the following we write a function to scrape the texts from URL from news outlets srf and 20min:

In [ ]:
# Function to scrape URL and extract text
def extract_text_from_url(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0"
        }
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            return None
        soup = BeautifulSoup(response.content, "html.parser")
        paragraphs = soup.find_all("p")
        text = " ".join(p.get_text().strip() for p in paragraphs)
        return text if text else None
    except Exception as e:
        print(f"Fehler bei URL: {url} → {e}")
        return None

# === 3. Filter: Nur Artikel ohne NZZ, mit gültiger URL, und leerem Text ===
df_scrape = df[
    (~df["newspaper_name"].str.lower().str.contains("nzz", na=False)) &
    (~df["news_article_URL"].isnull()) &
    (df["text"].isnull() | (df["text"].str.strip() == ""))
]

# === 4. Scraping starten und in bestehende 'text'-Spalte einfügen ===
for idx in tqdm(df_scrape.index):
    url = df.at[idx, "news_article_URL"]
    text = extract_text_from_url(url)
    df.at[idx, "text"] = text

# === 5. Neue Excel-Datei speichern (im gleichen Ordner) ===
output_path = "Data_ML_filled.xlsx"
df.to_excel(output_path, index=False)
print(f"✅ Fertig! Gespeichert als: {output_path}")





100%|██████████| 76/76 [00:33<00:00,  2.28it/s]

✅ Fertig! Gespeichert als: Data_ML_filled.xlsx


In [ ]:
# load new data
input_path = "Data_ML_filled.xlsx" 
df_new = pd.read_excel(input_path)


In [17]:
# show 5 random articles with text longer than 50 characters
df_sample = df[df["text"].notnull() & (df["text"].str.len() > 50)].sample(5, random_state=1)

# print the sample articles
pd.set_option('display.max_colwidth', None)
df_sample[["newspaper_name", "news_article_URL", "text"]]

newspaper_name  \
104          20min   
94           20min   
96             srf   
93             nzz   
105          20min   

                                                                                                                              news_article_URL  \
104                                https://www.20min.ch/story/abstimmung-darum-warnt-der-bundesrat-vor-der-biodiversitaetsinitiative-103171217   
94                                                   https://www.20min.ch/story/13-ahv-rente-linke-eroeffnen-den-abstimmungskampf-557637387047   
96   https://www.srf.ch/sendungen/kassensturz-espresso/kassensturz/abstimmung-am-3-maerz-2024-die-13-ahv-rente-sorgt-fuer-hitzige-diskussionen   
93                                         https://www.nzz.ch/schweiz/13-ahv-rente-die-linke-mobilisiert-gezielt-frauen-fuer-ein-ja-ld.1814584   
105                    https://www.20min.ch/story/chunsch-drus-darum-gehts-deshalb-spaltet-die-biodiversitaets-abstimmung-die-bauern-103167786   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

There are systematic .. problems in both 20min and srf articles which we now exclde from the dataset

In [35]:
# correct srf text by start and end
def clean_srf_text(text):
    if not isinstance(text, str):
        return text

    # 1. Liste störender Einleitungs-Absätze
    garbage_phrases = [
        "Einige Elemente auf SRF.ch funktionieren nur mit aktiviertem JavaScript",
        "SRF News App Bei Google Play",
        "In der App öffnen",
        "Für die Registrierung benötigen wir zusätzlich Angaben zu Ihrer Person."
    ]

    # 2. Entferne jeden dieser Sätze aus dem Text
    for phrase in garbage_phrases:
        text = text.replace(phrase, "")

    # Entferne Autorzeile: "Autor: \n Name Datum, Uhrzeit"
    text = re.sub(r"Autor:\s*\n?.*?\d{2}\.\d{2}\.\d{4}, \d{2}:\d{2}", "", text)

    # 3. Ende bei Kommentarbereich
    end = text.find("Bitte melden Sie sich an, um einen Kommentar zu erfassen")
    if end != -1:
        text = text[:end].strip()

    return text.strip()


# === 3. Nur SRF-Artikel bereinigen ===
mask_srf = df_new["newspaper_name"].str.lower().str.contains("srf", na=False)
df_new.loc[mask_srf, "text"] = df_new.loc[mask_srf, "text"].apply(clean_srf_text)

# === 4. Beispiel: 5 zufällige bereinigte SRF-Artikel anzeigen ===
pd.set_option('display.max_colwidth', None)  # zeigt ganzen Text

sample_srf = df_new[mask_srf & df_new["text"].notnull() & (df_new["text"].str.len() > 100)].sample(5, random_state=1)
sample_srf[["newspaper_name", "news_article_URL", "text"]]


newspaper_name  \
14            srf   
83            srf   
15            srf   
66            srf   
79            srf   

                                                                                                   news_article_URL  \
14  https://www.srf.ch/news/schweiz/masseneinwanderungs-initiative-die-auslaender-kommen-die-schweiz-unter-zugzwang   
83               https://www.srf.ch/radio-srf-1/abstimmung-am-13-februar-tierversuchsverbotsinitiative-ja-oder-nein   
15   https://www.srf.ch/news/schweiz/paedophilen-initiative-paedophile-sollen-nie-mehr-mit-kindern-arbeiten-duerfen   
66                  https://www.srf.ch/news/abstimmung-von-26-september-99-prozent-initiative-spaltet-die-meinungen   
79                  https://www.srf.ch/radio-srf-1/justizinitiative-richterwahl-im-losverfahren-die-bessere-loesung   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [36]:
## download the file to manually inspect:
df_new.to_excel("Data_ML_bereinigt.xlsx", index=False)

In [29]:
def fix_common_misencoding(text):
    replacements = {
        "â€™": "’",  # Fix curly apostrophe
        "â€˜": "‘",  # Fix left curly quote
        "â€”": "—",  # Fix em dash (if present)
        "â€œ": "“",  # Fix left double quote (if present)
        "â€": "”",   # Fix right double quote (if present)
    }
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    return text

In [31]:
df['Title'] = df['Title'].apply(fix_common_misencoding)

In [33]:
df.head()

,Title,Link,Text,Source,Bias
0,John Bolton’s Advice for the ‘Continuing Turmo...,https://thedispatch.com/article/john-boltons-a...,"With just days until his inauguration, it woul...",TheDispatch,lean right
1,Why Shrubland Makes Southern California’s Wild...,https://thedispatch.com/newsletter/techne/why-...,"Welcome back to Techne! In Star Trek, writers ...",TheDispatch,lean right
2,Los Angeles’ Destruction Was Fueled by Bad Pol...,https://thedispatch.com/newsletter/capitolism/...,"As fire still rages in the Los Angeles area, I...",TheDispatch,lean right
3,Acquiring Greenland Is a Good Idea. Threatenin...,https://thedispatch.com/article/acquiring-gree...,"Whether it was Will Rogers or Tony Soprano, th...",TheDispatch,lean right
4,Assessing Trump’s Expansionist Designs,https://thedispatch.com/newsletter/morning/ass...,"Happy Friday! If anyone was wondering, Belgium...",TheDispatch,lean right


In [34]:
df.shape

(3458, 5)